In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline

In [2]:
accre_cleaned =pd.read_csv('initial_data.csv')

In [3]:
accre_cleaned.shape

(3319256, 14)

In [4]:
accre_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3319256 entries, 0 to 3319255
Data columns (total 14 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       float64
USEDMEM      float64
REQTIME      int64
USEDTIME     int64
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
ID           int64
Sub          float64
dtypes: float64(3), int64(5), object(6)
memory usage: 354.5+ MB


In [5]:
accre_cleaned['EXITCODE'].value_counts()

0:0      3319151
0:9           46
1:0           31
7:0            9
0:125          6
0:15           4
2:0            4
0:12           3
11:0           1
0:6            1
Name: EXITCODE, dtype: int64

In [7]:
# keeping column with value for EXITCODE=0:0
accre_cleaned=accre_cleaned[accre_cleaned.EXITCODE=='0:0']

In [8]:
accre_cleaned.shape

(3319151, 14)

In [15]:
accre_cleaned= accre_cleaned.groupby(['ACCOUNT'])['REQMEM'].mean()

KeyError: 'Column not found: REQMEM'

,JOBID,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,ID,Sub
ACCOUNT,,,,,,,,,,,,,
alkmene,81963,81963,81963,81963,81963,81963,81963,81963,81963,81963,81963,81963,0
almonds,88643,88643,88643,88643,88643,88643,88643,88643,88643,88643,88643,88643,88596
amaranth,711,711,711,711,711,711,711,711,711,711,711,711,549
antares,178900,178900,178900,178900,178900,178900,178900,178900,178900,178900,178900,178900,0
arra,1315,1315,1315,1315,1315,1315,1315,1315,1315,1315,1315,1315,1247


In [ ]:
dummy=accre_cleaned.head(10000)

In [ ]:
*dummy.rolling(on)